In [3]:
import numpy as np 
import cv2
import os
from tensorflow.python.keras.preprocessing.image import img_to_array
from keras.models import model_from_json
from keras.preprocessing import image
os.chdir("C:/Users/Ruchita/Desktop/Project_Dataset/")

In [4]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

model_emotion = model_from_json(open("model_emotion.json" , "r").read())
model_emotion.load_weights("model_emotion.h5")

model_age = model_from_json(open("model_age.json" , "r").read())
model_age.load_weights("model_age.h5")

model_gender = model_from_json(open("model_gender.json" , "r").read())
model_gender.load_weights("model_gender.h5")

Emo_lable = ['Angry' , 'Disgust' , 'Fear' , 'Happy' , 'Sad' , 'Surprise' , 'Neutral']
age_lable = ['0-18 years' , '19-45 years' , '46-60 years' , 'above 60 years']
gen_lable = ['Male' , 'Female']

In [5]:
cap = cv2.VideoCapture(0)

while True:
    _ , image = cap.read()
    gray_image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)

    faces_detected = face_cascade.detectMultiScale(gray_image)
    for(x , y , w , h) in faces_detected:
        cv2.rectangle(image , (x , y) , (x+w , y+h) , (255 , 0 , 0))
        
        roi_gray = gray_image[y:y+w , x:x+h]
        roi_gray = cv2.resize(roi_gray , (48 , 48), interpolation=cv2.INTER_AREA)
        
        if np.sum([roi_gray])!=0:
            image_pixels = roi_gray.astype('float')# /255.0
            image_pixels = img_to_array(image_pixels)
            image_pixels = np.expand_dims(image_pixels , axis = 0)
     

            #emotion detection
            predictions = model_emotion.predict(image_pixels)
            max_index = max_index = Emo_lable[predictions.argmax()]
            label_position = (x,y)

            cv2.putText(image,max_index,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)

            #age
            predictions = model_age.predict(image_pixels)
            max_index = max_index = age_lable[predictions.argmax()]
            label_position = (x, y+h)

            cv2.putText(image,max_index,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            #gender
            predictions = model_gender.predict(image_pixels)
            max_index = max_index = gen_lable[predictions.argmax()]
            label_position = (x, y+h+35)

            cv2.putText(image,max_index,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            

        else:
            cv2.putText(image,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)



        cv2.imshow('Emotion_Age_Gender' , image)
    
    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>